# Exercice 1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

def discrete_sample(support_list, proba_list, N):
    c_list = [sum(proba_list[0:(i+1)]) for i in range(len(proba_list))]

    res = []

    for _ in range(N):
        k = 0
        U = np.random.rand()
        while (U > c_list[k] and k < len(proba_list)):
            k += 1
        res.append(support_list[k])

    return res

def one_sample_uniform(increased_support_list):
    k = 0
    U = np.random.rand()
    sum_proba = 1 / len(increased_support_list)
    while (U > sum_proba):
        k += 1
        sum_proba += 1 / len(increased_support_list)

    return increased_support_list[k]

def sample_abs_std_normal(N):
    res = []

    f = lambda x : 2 / np.sqrt(2*np.pi) * np.exp(-x**2/2) * int(x>0)
    g = lambda x : np.exp(-x) * int(x>0)
    inv_cdf_g = lambda x : -np.log(1-x) * int(x>0 and x<1)
    c = np.sqrt(2 * np.e / np.pi)

    for _ in range(N):
        while True:
            U = np.random.rand()

            U_ = np.random.rand()
            Y = inv_cdf_g(U_)

            if U < f(Y) / (c * g(Y)):
                break
        res.append(Y)
    
    return res

def sample_std_normal(N):
    return [one_sample_uniform([-1,1]) * i for i in sample_abs_std_normal(N)]

In [ ]:
# 1.

E = [1,2,3,4,5]

P = [[0.1, 0.1, 0.5, 0.2, 0.1],
     [0.3, 0.1, 0.1, 0.4, 0.1],
     [0.2, 0.1, 0.1, 0.5, 0.1],
     [0.1, 0.1, 0.1, 0.6, 0.1],
     [0.1, 0.2, 0.1, 0.4, 0.2]]

N = 100
# N = 100000
X0 = 1

chain = [X0]
for _ in range(N):
    chain.append(discrete_sample(E, P[chain[-1]-1], 1)[0])

plt.plot([n for n in range(N+1)], chain, marker='.', linestyle='None')

2.

$$\pi = (\pi_1, \pi_2, \pi_3, \pi_4, \pi_5)$$

$$\pi = \pi P, \sum_{k=1}^5 \pi_k = 1$$


Result from an online solver:
$$\pi = (0.1377, 0.1111, 0.1551, 0.4849, 0.1111)$$

In [ ]:
# 3.

pi_est = [
    sum([
        int(x == e)
        for x in chain
    ]) / N
    for e in E
]

pi_est

In [ ]:
# 4.

transition_count = [[0,0,0,0,0],
                    [0,0,0,0,0],
                    [0,0,0,0,0],
                    [0,0,0,0,0],
                    [0,0,0,0,0]]
start_state_count = [0,0,0,0,0]

for i in range(N):
    transition_count[chain[i]-1][chain[i+1]-1] += 1
    start_state_count[chain[i]-1] += 1

P_est = [
    [
        transition_count[i][j] / start_state_count[i]
        for j in range(len(transition_count[i]))
    ]
    for i in range(len(transition_count))
]

P_est

# Exercice 2

In [ ]:
# 1.

d = 11
A_list = [-0.05+0.01*k for k in range(d)]
p_list = [1/d]*d
r = 2

n = 1000
N = 10

X_lists = []
for _ in range(N):
    X_lists.append([r])
    for _ in range(n):
        X_lists[-1].append(X_lists[-1][-1] + discrete_sample(A_list, p_list, 1)[0])
    plt.plot([i for i in range(n+1)], X_lists[-1])

2.

$$\frac{1}{n} \sum_{k=1}^{n} \mathbb{E} \left[ (X_k-r)^2 \right] = \mathbb{E} \left[ \frac{1}{n} \sum_{k=1}^{n} (X_k-r)^2 \right]$$

In [ ]:
# 2.

est = sum([ 
    1/n * sum([
        (X_lists[i][k] - r)**2 for k in range(n+1)
    ]) 
    for i in range(N)
]) / N

est

3. To find the confidence interval, the standard deviation can be computed using:

$$S^2_{N,g} = \frac{1}{N-1} \sum_{k=1}^N \left( g(X_0^{(k)},...,X_n^{(k)}) - M_N(g(X_0,...,X_n)) \right)^2$$

# Exercice 3

In [ ]:
# 1.

N = 10
X0 = 4
E = [i for i in range(N+1)]
# m = 10**5
m = 100

transition_proba = lambda Xn, Xn1: math.factorial(N)/(math.factorial(Xn1)*math.factorial(N-Xn1)) * (Xn/N)**Xn1 * (1-Xn/N)**(N-Xn1)

chain = [X0]
for _ in range(m):
    chain.append(
        discrete_sample(
            E, 
            list(map(transition_proba, [chain[-1]]*(N+1), E)), 
            1
        )[0]
    )

plt.plot([i for i in range(m+1)], chain)

In [ ]:
transition_count = [[0]*(N+1)]*(N+1)
start_state_count = [0]*(N+1)

for i in range(m):
    transition_count[chain[i]][chain[i+1]] += 1
    start_state_count[chain[i]] += 1

P_est = [
    [
        # condition to avoid division by zero
        (transition_count[i][j] / start_state_count[i]) if start_state_count[i] != 0 else 0
        for j in range(len(transition_count[i]))
    ]
    for i in range(len(transition_count))
]

P_est

# Exercice 6

In [ ]:
d = 11
A = [-0.05 + 0.01*k for k in range(d)]
P = [1/d] * d
r = 2
x0 = r
n = 1000

# 1.

delta = 0.05

X = [x0]
Y = [x0 + delta*sample_std_normal(1)[0]]

for _ in range(n):
    X.append(X[-1] + discrete_sample(A, P, 1)[0])
    Y.append(X[-1] + delta*sample_std_normal(1)[0])

plt.plot([i for i in range(n+1)], X)
plt.plot([i for i in range(n+1)], Y)